In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

AttributeError: module 'pandas' has no attribute 'core'

In [1]:
#マウスリストの作成、神経データが記録されている全フェーズリストの作成
mouse_list = ["ID138464-1","ID138464-2","ID138464-3","ID138464-4","ID138464-5","ID138464-6"]
phaseall_list=["HC1","B1","B2","HC2","A1","A2","A31","A32","A41","A42","HC3","A5","B3","A6","A7","HC4","B4"]

#対象マウス決定

for m in range(0,len(mouse_list)):

    mouse = mouse_list[m]

    #神経データをデータフレームにインポートする
    inputf='%s/python/%s_Longitudinal_Traces.csv' %(mouse,mouse)
    dff=pd.read_csv(inputf,header=0)
    dff0=pd.DataFrame()

    #Acceptのみを抽出したデータフレームの作成
    dff0=dff.iloc[0,:]
    Accept=dff0.index[dff0==' accepted']
    dfa=pd.DataFrame()
    dfa['Time2']=dff.iloc[1:,0]
    for accept_c in range (0,len(Accept)):
        dfa[Accept[accept_c]]=dff[Accept[accept_c]].iloc[1:]
        dfa=dfa.astype('float')

    #Acceptcellのfirst cellとlast cellを抽出
    cell_f=dfa.columns[1]
    cell_l=dfa.columns[dfa.shape[1]-1]

    #レコード間の時間差からphaseを割り当てる
    dfa['dt']=dfa['Time2'].diff()
    dfa['dt'].iloc[0]=0.05
    dfa['phase']=-1
    phase=0
    for i in range(0,dfa.shape[0]):    
        if(dfa['dt'].iloc[i]>10):        
            phase=phase+1
        dfa.iloc[i,dfa.columns.get_loc('phase')]=phaseall_list[phase]

    #割り当てたphaseごとにcsvを生成(XYdataがphaseごとに生成されているため)
    for phase in range(0,len(phaseall_list)):    
        outputf='%s/python/%s_%s_Long_Traces.csv' %(mouse,mouse,phaseall_list[phase])    
        df_phase=dfa[dfa['phase']==phaseall_list[phase]]        
        df_phase.to_csv(outputf,index=False)

    #XYデータにヘッダー情報がたくさん入っている、かつcsvなのでヘッダー情報をカラム情報以外取り除き、csvに出力する
    phasepre_list=["B1","B2","A1","A2","A3","A4","A5","B3","A6","A7","B4"]
    for phase_pre in range(0,len(phasepre_list)):
        inputXY='%s/python/%s_XYdata/%s_%s_%s_XY.txt' %(mouse,mouse,mouse,phase_pre+1,phasepre_list[phase_pre])
        outputXY='%s/python/%s_XYdata/%s_%s_XY.csv' %(mouse,mouse,mouse,phasepre_list[phase_pre])
        dfxyp=pd.read_table(inputXY)
        h=dfxyp[dfxyp['TimeFZ4']=='Slice'].index[0]
        dfxy=pd.read_table(inputXY,header=h+1)
        dfxy.to_csv(outputXY,index=False)

    #XYデータがA3,A4を前半後半に分割していないので分割してからcsvに
    phasetransb_list=["A3","A4"]
    phasetransa_list=["A31","A32","A41","A42"]
    for phase_b in range(0,2):
        inputXY='%s/python/%s_XYdata/%s_%s_%s_XY.txt' %(mouse,mouse,mouse,phase_b+5,phasetransb_list[phase_b])
        for phase_a in range(0, 2):
            outputXY='%s/python/%s_XYdata/%s_%s_XY.csv' %(mouse,mouse,mouse,phasetransa_list[2*phase_b+phase_a])
            dfxyp=pd.read_table(inputXY)
            h=dfxyp[dfxyp['TimeFZ4']=='Slice'].index[0]
            dfxy=pd.read_table(inputXY,header=h+1)
            dfxy.iloc[3000*phase_a+0:3000*phase_a+621,:].to_csv(outputXY,index=False)

    #分析対象となるphase_listを作成
    phase_list=["A1","A2","A31","A32","A41","A42","A5","A6","A7"]

    #後ほど使う全phase合わせた用のデータフレーム
    dfmeandiff_all=pd.DataFrame()

    #phaseごとに分析をする
    for ph in range(0,len(phase_list)):
        phase=phase_list[ph]
        inputXY='%s/python/%s_XYdata/%s_%s_XY.csv' %(mouse,mouse,mouse,phase)
        dfx = pd.read_csv(inputXY,header=0)
        inputT='%s/python/%s_%s_Long_Traces.csv' %(mouse,mouse,phase)
        dfl=pd.read_csv(inputT,header=0)

        #FZ-nonFZ状態の入れ替わりからsessionを付与
        dfx["session"]=0
        ses=dfx.columns.get_loc('session')
        for i in range(1,dfx.shape[0]):
            if (dfx.iloc[i].loc["FZ"]!=dfx.iloc[i-1].loc["FZ"]):
                dfx.iloc[i,ses]= int(dfx.iloc[i-1,ses])+1
            else:
                dfx.iloc[i,ses]= int(dfx.iloc[i-1,ses])

        #後ほどFZの短いセッション結合を行うための準備
        dfx['FZ_c0']=dfx['FZ']
        dfx['ses_c0']=dfx['session']
        ses_cnt=0
        FZ_col=dfx.columns.get_loc('FZ_c0')
        session_col=dfx.columns.get_loc('ses_c0')
        ses_c='ses_c%s'%(ses_cnt)
        mconseq='mconseq_%s'%(ses_cnt)
        dfx[mconseq]=0
        mconseq_col=dfx.columns.get_loc('mconseq_0')
        Slice_col=dfx.columns.get_loc('Slice')

        #mconseq(マイナス方向に伸ばしたconseqFZ)を付与
        for i in range (0,dfx.shape[0]):
            ses_no = dfx.iloc[i,session_col]
            if dfx.iloc[i,FZ_col] == 0:
                if ses_no == dfx.iloc[dfx.shape[0]-1].loc[ses_c]:
                    dfx.iloc[i,mconseq_col]=-100
                    break;
                df_ns=dfx[dfx[ses_c]==ses_no+1]
            else:
                df_ns=dfx[dfx[ses_c]==ses_no]
            df_ns1=df_ns.iloc[0]
            dfx.iloc[i,mconseq_col]=int(dfx.iloc[i,Slice_col]-df_ns1.iloc[Slice_col]+1)

        #validラベルを付与
        valid_fz='valid_%s'%(ses_cnt)
        dfx[valid_fz]=0
        valid_col=dfx.columns.get_loc(valid_fz)
        for i in range(0,dfx.shape[0]):
            ses_no_v = dfx.iloc[i,session_col]
            if dfx.iloc[i,FZ_col] == 0:
                if ses_no_v == dfx.iloc[dfx.shape[0]-1].loc[ses_c]:
                    break;
                df_ns_v=dfx[dfx[ses_c]==ses_no_v+1]
            else:
                df_ns_v=dfx[dfx[ses_c]==ses_no_v]
            df_ns1_v=df_ns_v.iloc[df_ns_v.shape[0]-1]
            if df_ns1_v[mconseq_col]<9:
                dfx.iloc[i,valid_col]=0
            else:
                dfx.iloc[i,valid_col]=1

        #短いFZセッションの結合→mconseqを付与→validを付与
        for ses_cnt in range(1,10):
            FZ_c='FZ_c%s'%(ses_cnt)
            FZ_cm='FZ_c%s'%(ses_cnt-1)
            ses_c='ses_c%s'%(ses_cnt)
            ses_cm='ses_c%s'%(ses_cnt-1)
            dfx[FZ_c]=dfx[FZ_cm]
            dfx[ses_c]=0
            fzc=dfx.columns.get_loc(FZ_c)
            cnt=0
            sup=0
            r=20
            for i in range(1,int(dfx.iloc[dfx.shape[0]-1].loc[ses_cm])+1):
                fz=dfx[dfx[ses_cm]==i].iloc[0,fzc]
                nfz=int((1-fz)*(1-fz))
                slice_b=dfx[dfx[ses_cm]==i].iloc[0].loc['Slice']
                slice_a=dfx[dfx[ses_cm]==i].iloc[dfx[dfx[ses_cm]==i].shape[0]-1].loc['Slice']
                Queryp='%d<=Slice<%d'%(int(slice_b),int(slice_a+r))
                Querym='%d<=Slice<%d'%(int(slice_b),int(slice_a+r))
                dfx_sp=dfx.query(Queryp)
                dfx_sm=dfx.query(Querym)
                if(i<int(dfx.iloc[dfx.shape[0]-1].loc[ses_cm])):
                    pp=1.0*dfx_sp[FZ_cm].value_counts()[fz]/(dfx_sp.shape[0]*1.0)
                    pm=1.0*dfx_sm[FZ_cm].value_counts()[fz]/(dfx_sm.shape[0]*1.0)
                else:
                    p=1
                if((dfx[dfx[ses_cm]==i].shape[0]<10) and (pp<0.350 or pm<0.350) and  (fz==0)):
                    dfx.loc[dfx[ses_cm]==i,ses_c]=cnt
                    dfx.loc[dfx[ses_cm]==i,FZ_c]=nfz
                    sup=1
                else:
                    cnt+=1-sup
                    dfx.loc[dfx[ses_cm]==i,ses_c]=cnt
                    dfx.loc[dfx[ses_cm]==i,FZ_c]=fz
                    sup=0
            mconseq='mconseq_%s'%(ses_cnt)
            dfx[mconseq]=-10
            FZ_col=dfx.columns.get_loc(FZ_c)
            session_col=dfx.columns.get_loc(ses_c)
            mconseq_col=dfx.columns.get_loc(mconseq)
            Slice_col=dfx.columns.get_loc('Slice')
            for i in range (0,dfx.shape[0]):
                ses_no = dfx.iloc[i,session_col]
                if dfx.iloc[i,FZ_col] == 0:
                    if ses_no == dfx.iloc[dfx.shape[0]-1].loc[ses_c]:
                        break;
                    df_ns=dfx[dfx[ses_c]==ses_no+1]
                else:
                    df_ns=dfx[dfx[ses_c]==ses_no]
                df_ns1=df_ns.iloc[0]
                dfx.iloc[i,mconseq_col]=int(dfx.iloc[i,Slice_col]-df_ns1.iloc[Slice_col]+1)
            valid_fz='valid_%s'%(ses_cnt)
            dfx[valid_fz]=0
            valid_col=dfx.columns.get_loc(valid_fz)
            for i in range(0,dfx.shape[0]):
                ses_no_v = dfx.iloc[i,session_col]
                if dfx.iloc[i,FZ_col] == 0:
                    if ses_no_v == dfx.iloc[dfx.shape[0]-1].loc[ses_c]:
                        break;
                    df_ns_v=dfx[dfx[ses_c]==ses_no_v+1]
                else:
                    df_ns_v=dfx[dfx[ses_c]==ses_no_v]
                df_ns1_v=df_ns_v.iloc[df_ns_v.shape[0]-1]
                if df_ns1_v[mconseq_col]<9:
                    dfx.iloc[i,valid_col]=0
                else:
                    dfx.iloc[i,valid_col]=1
            if(dfx.iloc[dfx.shape[0]-1].loc[ses_cm]==dfx.iloc[dfx.shape[0]-1].loc[ses_c]):
                break;

        #ここで神経データと行動データを結合       
        dfx_m=pd.DataFrame(columns=dfx.columns,index=np.arange(0,int(dfx.iloc[dfx.shape[0]-1].loc['Slice']-1)*10))
        for i in range(2,int(dfx.shape[0])):
            for j in range(0,10):
                for k in range(0,len(dfx_m.columns)):
                    dfx_m.iloc[(i-2)*10+j].loc[dfx_m.columns[k]]=dfx.iloc[i].loc[dfx_m.columns[k]]
        df=pd.concat([dfx_m, dfl], axis=1)
        df=df.dropna(how='any')
        outputf='%s/python/%s_%s_con.csv' %(mouse,mouse,phase)          
        df.to_csv(outputf,index=False)

        #Sliceごとに平均をとったdfmeanの作成
        Index=np.arange(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1),1)
        df_mean=pd.DataFrame(index=Index,columns=[])
        for i in range(0,len(df.columns)):
            df_mean[df.columns[i]]=-100
        for i_n in range(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1)):
            i = i_n - int(df.iloc[0].loc['Slice'])
            df_mean.iloc[i].loc['Slice']=i_n
            dfs = df[df['Slice']==i_n]
            for k in range(0,df.columns.get_loc('Time2')+1):
                df_mean.iloc[i,k]=dfs.iloc[0,k]
            for j in range(dfs.columns.get_loc(cell_f),dfs.columns.get_loc(cell_l)+1):
                df_mean.iloc[i,j]=dfs[dfs.columns[j]].mean()
            for l in range(dfs.columns.get_loc('session'),dfs.columns.get_loc(ses_c)+1):
                df_mean.iloc[i,l]=int(dfs.iloc[0,l])
        dfmean=df_mean
        outputf='%s/python/%s_%s_mean.csv' %(mouse,mouse,phase)          
        dfmean.to_csv(outputf,index=False)
        
                #sliceiniの情報をdiff化→std化
        dfmeandiff=df_mean.diff()
        dfmeandiff.loc[:,:'Time2']=df_mean.loc[:,:'Time2']
        dfmeandiff=dfmean.dropna(how='any')
        dfmeandiff_std=pd.DataFrame(columns=dfmeandiff.columns,index=dfmeandiff.index)
        for k in range(dfmeandiff_std.columns.get_loc('Slice'),dfmeandiff_std.columns.get_loc(cell_f)):
            dfmeandiff_std[dfmeandiff_std.columns[k]]=dfmeandiff[dfmeandiff_std.columns[k]]
        for i in range(dfmeandiff_std.columns.get_loc(cell_f),dfmeandiff_std.columns.get_loc(cell_l)+1):
            for j in range(0,dfmeandiff_std.shape[0]):
                dfmeandiff_std.iloc[j,i]=((dfmeandiff.iloc[j,i]-dfmeandiff[dfmeandiff_std.columns[i]].mean())/dfmeandiff[dfmeandiff_std.columns[i]].std())
        dfmeandiff_all=pd.concat([dfmeandiff_all,dfmeandiff_std])
    outputf='%s/python/%s_meandiffall.csv' %(mouse,mouse)          
    dfmeandiff_all.to_csv(outputf,index=False)

AttributeError: module 'pandas' has no attribute 'read_csv'

In [ ]:
        #セッション結合操作の前のセッションで、sliceini「X:Time1,Y:Ca2+img強度」
        for j in range(dfmean.columns.get_loc(cell_f),dfmean.columns.get_loc(cell_l)+1):
            plt.figure(figsize=(5.4, 3.6), dpi=80)
            plt.plot(dfmean['Slice'],dfmean[dfmean.columns[j]],color='red')
            plt.ylim(-0.05,0.30)
            for i in range(0,dfmean.shape[0]):
                if dfmean.iloc[i].loc['FZ'] == 1:
                    if dfmean.iloc[i].loc['X']  <= 125:
                        if dfmean.iloc[i].loc['Y']  <=100:
                            Color='lightpink'
                        else:
                            Color='lightblue'
                    else:
                        if dfmean.iloc[i].loc['Y']  <=100:
                            Color='silver'
                        else:
                            Color='lightgreen'
                    plt.fill([dfmean.iloc[i].loc['Slice'],dfmean.iloc[i].loc['Slice']+1,dfmean.iloc[i].loc['Slice']+1,dfmean.iloc[i].loc['Slice']],[-0.05,-0.05,0.30,0.30],color=Color)
                if (dfmean.iloc[i].loc['Shock'] == 1):
                    plt.plot([dfmean.iloc[i].loc['Slice'],dfmean.iloc[i].loc['Slice']],[-0.05,0.30],color='red',linestyle='dashed')
            plt.xlabel('Slice')
            plt.ylabel('Ca2+Img')
            title_name='sliceini_%s_%s' %(phase,dfmean.columns[j])
            plt.title(title_name)
            file_name='%s/python/fig/'%(mouse)+title_name+'.png'
            plt.savefig(file_name)

        #セッション結合操作後のセッションで、sliceini「X:Time1,Y:Ca2+img強度」
        for j in range(dfmean.columns.get_loc(cell_f),dfmean.columns.get_loc(cell_l)+1):
            plt.figure(figsize=(5.4, 3.6), dpi=80)
            plt.plot(dfmean['Slice'],dfmean[dfmean.columns[j]],color='red')
            plt.ylim(-0.05,0.30)
            for i in range(0,dfmean.shape[0]):
                if dfmean.iloc[i].loc[FZ_c] == 1:
                    if dfmean.iloc[i].loc['X']  <= 125:
                        if dfmean.iloc[i].loc['Y']  <=100:
                            Color='lightpink'
                        else:
                            Color='lightblue'
                    else:
                        if dfmean.iloc[i].loc['Y']  <=100:
                            Color='silver'
                        else:
                            Color='lightgreen'
                    plt.fill([dfmean.iloc[i].loc['Slice'],dfmean.iloc[i].loc['Slice']+1,dfmean.iloc[i].loc['Slice']+1,dfmean.iloc[i].loc['Slice']],[-0.05,-0.05,0.30,0.30],color=Color)
                if (dfmean.iloc[i].loc['Shock'] == 1):
                    plt.plot([dfmean.iloc[i].loc['Slice'],dfmean.iloc[i].loc['Slice']],[-0.05,0.30],color='red',linestyle='dashed')
            plt.xlabel('Slice')
            plt.ylabel('Ca2+Img')
            title_name='sliceope_%s_%s' %(phase,dfmean.columns[j])
            file_name='%s/python/fig/'%(mouse)+title_name+'.png'
            plt.title(title_name)
            plt.savefig(file_name)

        #セッション結合後のヒストグラム(X:img強度,Y:密度)
        for j in range(dfmean.columns.get_loc(cell_f),dfmean.columns.get_loc(cell_l)+1):
            plt.figure(figsize=(5.4, 3.6), dpi=80)
            df_m=dfmean[(dfmean[mconseq]<0)]
            df_p=dfmean[(dfmean[mconseq]>=0) & (dfmean[valid_fz]==1)]
            plt.hist([df_m[dfmean.columns[j]],dfmean[dfmean.columns[j]],df_p[dfmean.columns[j]]],bins=10,normed=True,label=['nonFZ','ALL','FZ'])
            plt.xlabel('Ca2+img')
            plt.ylabel('density')
            title_name='histope_%s_%s' % (phase,dfmean.columns[j])
            file_name='%s/python/fig/'%(mouse)+title_name+'.png'
            plt.title(title_name)
            plt.savefig(file_name)

        #セッション結合前のヒストグラム(X:img強度,Y:密度)
        for j in range(df.columns.get_loc(cell_f),df.columns.get_loc(cell_l)+1):   
            plt.figure(figsize=(5.4, 3.6), dpi=80)
            df_m=dfmean[(dfmean['mconseq_0']<0)]
            df_p=dfmean[(dfmean['mconseq_0']>=0) & (dfmean['valid_0']==1)]
            plt.hist([df_m[dfmean.columns[j]],dfmean[dfmean.columns[j]],df_p[dfmean.columns[j]]],bins=10,normed=True,label=['nonFZ','ALL','FZ'])
            plt.xlabel('Ca2+img')
            plt.ylabel('density')
            title_name='histini_%s_%s' % (phase,dfmean.columns[j])
            file_name='%s/python/fig/'%(mouse) +title_name+'.png'
            plt.title(title_name)
            plt.savefig(file_name)

In [ ]:
        for j in range(dfmeandiff.columns.get_loc(cell_f),dfmeandiff.columns.get_loc(cell_l)+1):
            plt.figure(figsize=(5.4, 3.6), dpi=80)
            plt.plot(dfmeandiff_std['Slice'],dfmeandiff_std[dfmeandiff.columns[j]],color='black')
            for i in range(0,dfmeandiff.shape[0]):
                if dfmeandiff.iloc[i].loc['FZ'] == 1:
                    if dfmeandiff.iloc[i].loc['X']  <= 125:
                        if dfmeandiff.iloc[i].loc['Y']  <=100:
                            Color='lightpink'
                        else:
                            Color='lightblue'
                    else:
                        if dfmeandiff.iloc[i].loc['Y']  <=100:
                            Color='silver'
                        else:
                            Color='lightgreen'
                    plt.fill([dfmeandiff.iloc[i].loc['Slice'],dfmeandiff.iloc[i].loc['Slice']+1,dfmeandiff.iloc[i].loc['Slice']+1,dfmeandiff.iloc[i].loc['Slice']],[-5.0,-5.0,10.0,10.0],color=Color)
                if (dfmeandiff.iloc[i].loc['Shock'] == 1):
                    plt.plot([dfmeandiff.iloc[i].loc['Slice'],dfmeandiff.iloc[i].loc['Slice']],[-5.0,10.0],color='red',linestyle='dashed')
            plt.plot([dfmeandiff.iloc[0].loc['Slice'],dfmeandiff.iloc[dfmeandiff.shape[0]-1].loc['Slice']],[5.0,5.0],color='red',linewidth=0.5)
            plt.plot([dfmeandiff.iloc[0].loc['Slice'],dfmeandiff.iloc[dfmeandiff.shape[0]-1].loc['Slice']],[4.0,4.0],color='red',linewidth=0.5)
            plt.plot([dfmeandiff.iloc[0].loc['Slice'],dfmeandiff.iloc[dfmeandiff.shape[0]-1].loc['Slice']],[3.0,3.0],color='red',linewidth=0.5)
            plt.plot([dfmeandiff.iloc[0].loc['Slice'],dfmeandiff.iloc[dfmeandiff.shape[0]-1].loc['Slice']],[2.0,2.0],color='red',linewidth=0.5)
            plt.plot([dfmeandiff.iloc[0].loc['Slice'],dfmeandiff.iloc[dfmeandiff.shape[0]-1].loc['Slice']],[1.0,1.0],color='red',linewidth=0.5)
            plt.ylim(-5.0,10.0)
            plt.xlabel('Slice')
            plt.ylabel('Ca2+Img(std)')        
            title_name='slicediffini_%s_%s' % (phase,dfmean.columns[j])
            plt.title(title_name)
            file_name='%s/python/fig/'%(mouse) +title_name+'.png'
            plt.savefig(file_name)

        #trajectoryを記録
        #plt.plot(df["X"],df["Y"],marker="o",color="blue")
        plt.show()
        plt.figure()
        plt.ylim(0,200)
        plt.xlim(0,250)
        plt.plot(df["X"],df["Y"],marker="o",color="blue")
        df_FZ=df[df["FZ"]!=0]
        df_nFZ=df[df["FZ"]==0]
        title_name='trajectory_%s' % (phase)
        plt.title(title_name)
        file_name='%s/python/fig/'%(mouse)+title_name+'.png'
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.savefig(file_name)

        #FZpointを記録
        plt.figure()
        plt.scatter(df_FZ["X"],df_FZ["Y"],marker="o",color="blue")
        plt.ylim(0,200)
        plt.xlim(0,250)
        title_name='FZpoint_%s' % (phase)
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title(title_name)
        file_name='%s/python/fig/'%(mouse)+title_name+'.png'
        plt.savefig(file_name)

        #shockのphaseにおいて、shock_pointを記録
        if phase=='A2' or phase=='A6':
            plt.figure()
            df_shock=df[df['Shock']==1]
            plt.scatter(df_shock["X"],df_shock["Y"],marker="o",color="red")
            for i in range(0,df_shock.shape[0]):
                plt.annotate(int(df_shock.iloc[i].loc["Slice"]), (df_shock.iloc[i].loc["X"], df_shock.iloc[i].loc["Y"]),size=10)
            plt.xlim(0,250)
            plt.ylim(0,200)
            title_name='shock_point_%s' % (phase)
            plt.title(title_name)
            file_name='%s/python/fig/'%(mouse)+title_name+'.png'
            plt.savefig(file_name)

            #shockのphaseにおいて、histshockを記録
            shocked_time=df_shock.iloc[0].loc['Slice']
            for j in range(df.columns.get_loc(cell_f),df.columns.get_loc(cell_l)+1):   
                df_b=df[(df['Slice']<shocked_time)]
                df_a=df[(df['Slice']>=shocked_time)]
                plt.hist([df_b[df.columns[j]],df_a[df.columns[j]]],bins=10,normed=True,label=['beforeSHOCK','afterSHOCK'])
                plt.xlabel('Ca2+img')
                plt.ylabel('density')
                #plt.ylim(0,5)
                title_name='histshock_%s_%s' % (phase,df.columns[j])
                plt.title(title_name)
                file_name='%s/python/fig/'%(mouse)+title_name+'.png'
                plt.savefig(file_name)

    #heatmapの作成

    dfmap=pd.DataFrame(index=np.arange(195,-5,-200/20),columns=np.arange(6.25,256.25,250/20))
    dfmap_n=pd.DataFrame(index=np.arange(195,-5,-200/20),columns=np.arange(6.25,256.25,250/20))
    dfmap_d=pd.DataFrame(index=np.arange(195,-5,-200/20),columns=np.arange(6.25,256.25,250/20))

    for i in range(0,20):

        for j in range(0,20):

            n=dfmeandiff_all[(dfmeandiff_all['X']>float(dfmap.columns[i]-6.25) )& (dfmeandiff_all['X']<float(dfmap.columns[i]+6.25) )&( dfmeandiff_all['Y']>float(dfmap.index[j]-5) )& (dfmeandiff_all['Y']<float(dfmap.index[j]+5))].shape[0]

            dfmap_n.iloc[i,j]=n


    for ts in range(1,5):

        for cell in range(dfmeandiff_all.columns.get_loc(cell_f),dfmeandiff_all.columns.get_loc(cell_l)+1):

            for i in range(0,20):

                for j in range(0,20):

                    if dfmap_n.iloc[i,j]<1.0:


                        dfmap.iloc[i,j]=0

                        dfmap_d.iloc[i,j]=0

                    else:

                        d=dfmeandiff_all[(dfmeandiff_all['X']>float(dfmap.columns[i]-6.25)) & (dfmeandiff_all['X']<float(dfmap.columns[i]+6.25)) & (dfmeandiff_all['Y']>float(dfmap.index[j]-5)) & (dfmeandiff_all['Y']<float(dfmap.index[j]+5) )& (dfmeandiff_all[dfmeandiff_all.columns[cell]]>ts)].shape[0]


                        dfmap_d.iloc[i,j]=d

                        dfmap.iloc[i,j]=(d/dfmap_n.iloc[i,j])

            dfmap=dfmap.astype(float)
            #dfmap.columns.astype(float)
            #dfmap.index.astype(float)

            title_name='heatdiffprob_ts%s_%s' % (ts,dfmeandiff_all.columns[cell])
            file_name='%s/python/fig/'%(mouse)+title_name+'.png'
            plt.title(title_name)
            plt.xlabel('x')
            plt.ylabel('y')
            sns.heatmap(dfmap)
            plt.savefig(file_name)

            dfmap_d=dfmap_d.astype(float)
            title_name='heatdiff_ts%s_%s' % (ts,dfmeandiff_all.columns[cell])
            file_name='%s/python/fig/'%(mouse)+title_name+'.png'
            plt.title(title_name)
            plt.xlabel('x')
            plt.ylabel('y')
            sns.heatmap(dfmap_d)
            plt.savefig(file_name)

/Users/rentakakura/anaconda/envs/jupyter-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/rentakakura/anaconda/envs/jupyter-env/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/rentakakura/anaconda/envs/jupyter-env/lib/python3.6/site-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value i